# Spark Walmart Data Analysis Project Exercise 

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [2]:
import findspark
findspark.init('/home/jubinsoni/spark-2.1.0-bin-hadoop2.7')

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('walmart').getOrCreate()

FileNotFoundError: [WinError 2] The system cannot find the file specified

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [ ]:
df = spark.read.csv('walmart_stock.csv', inferSchema=True, header=True)

#### What are the column names?

In [ ]:
df.columns

#### What does the Schema look like?

In [ ]:
df.printSchema()

#### Print out the first 5 columns.

In [ ]:
for line in df.head(5):
    print(line, '\n')

#### Use describe() to learn about the DataFrame.

In [ ]:
df.describe().show()

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [ ]:
'''
from pyspark.sql.types import (StructField, StringType,
                               IntegerType, StructType)

data_schema = [StructField('summary', StringType(), True),
               StructField('Open', StringType(), True),
               StructField('High', StringType(), True),
               StructField('Low', StringType(), True),
               StructField('Close', StringType(), True),
               StructField('Volume', StringType(), True),
               StructField('Adj Close', StringType(), True)
              ]

final_struc = StructType(fields=data_schema)

'''
df = spark.read.csv('walmart_stock.csv', inferSchema=True, header=True)

df.printSchema()
#The schema given below is wrong, as it is mostly from an older version.
#Spark is able to predict the schema correctly now

In [ ]:
from pyspark.sql.functions import format_number

summary = df.describe()
summary.select(summary['summary'],
                  format_number(summary['Open'].cast('float'), 2).alias('Open'),
                  format_number(summary['High'].cast('float'), 2).alias('High'),
                  format_number(summary['Low'].cast('float'), 2).alias('Low'),
                  format_number(summary['Close'].cast('float'), 2).alias('Close'),
                  format_number(summary['Volume'].cast('int'),0).alias('Volume')
                 ).show()

#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [ ]:
df_hv = df.withColumn('HV Ratio', df['High']/df['Volume']).select(['HV Ratio'])
df_hv.show()

#### What day had the Peak High in Price?

In [ ]:
df.orderBy(df['High'].desc()).select(['Date']).head(1)[0]['Date']

#### What is the mean of the Close column?

In [ ]:
from pyspark.sql.functions import mean

df.select(mean('Close')).show()

#### What is the max and min of the Volume column?

In [ ]:
from pyspark.sql.functions import min, max

df.select(max('Volume'),min('Volume')).show()

#### How many days was the Close lower than 60 dollars?

In [ ]:
df.filter(df['Close'] < 60).count()

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [ ]:
df.filter('High > 80').count() * 100/df.count()

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [ ]:
df.corr('High', 'Volume')

In [ ]:
from pyspark.sql.functions import corr

df.select(corr(df['High'], df['Volume'])).show()

#### What is the max High per year?

In [ ]:
from pyspark.sql.functions import (dayofmonth, hour,
                                   dayofyear, month,
                                   year, weekofyear,
                                  format_number, date_format)

year_df = df.withColumn('Year', year(df['Date']))

year_df.groupBy('Year').max()['Year', 'max(High)'].show()

#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [ ]:
#Create a new column Month from existing Date column
month_df = df.withColumn('Month', month(df['Date']))

#Group by month and take average of all other columns
month_df = month_df.groupBy('Month').mean()

#Sort by month
month_df = month_df.orderBy('Month')

#Display only month and avg(Close), the desired columns
month_df['Month', 'avg(Close)'].show()

# Thank you!